In [1]:
from pathlib import Path

import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

from astropy import units as u
from astropy import table

import pyvo as vo

from tqdm.notebook import tqdm

from multiprocess import Pool

from dill import dump, load

In [2]:
import warnings
from astroquery.exceptions import NoResultsWarning
warnings.simplefilter('ignore', NoResultsWarning)
warnings.simplefilter('ignore', u.UnitsWarning) # comes up in the parsing of some heasarc tables but is just a "discouraged by FITS standard" issue

In [3]:
import SAGA
saga = SAGA.QuickStart('/Users/etollerud/Dropbox/', '/../')

SAGA `shared_dir` set to /Users/etollerud/Dropbox/
SAGA `local_dir`  set to /../


In [4]:
hosts = saga.host_catalog.load()

# Try using the HEASARC Xamin interface

In [12]:
from astroquery.heasarc import Xamin

CONCURRENCY_QRYS = 10

In [11]:
tables = Xamin.tables()
list(tables.items())[:20]

[('TAP_SCHEMA.columns', 'Available columns'),
 ('TAP_SCHEMA.key_columns', 'Key/Foreign key pairs -- usused'),
 ('TAP_SCHEMA.keys', 'Joins between tables -- unused'),
 ('TAP_SCHEMA.schemas', 'Available schemas'),
 ('TAP_SCHEMA.tables', 'Available tables'),
 ('"first"', 'Faint Images of the Radio Sky at Twenty cm (FIRST)'),
 ('a1', 'HEAO 1 A1 X-Ray Source Catalog'),
 ('a1point', 'HEAO 1 A1 Lightcurves'),
 ('a2lcpoint', 'HEAO 1 A2 Pointed Lightcurves'),
 ('a2lcscan', 'HEAO 1 A2 Scanned Lightcurves'),
 ('a2led', 'HEAO 1 A2 LED Catalog'),
 ('a2pic', 'HEAO 1 A2 Piccinotti Catalog'),
 ('a2point', 'HEAO 1 A2 Pointing Catalog'),
 ('a2rtraw', 'HEAO 1 A2 Raw Rates'),
 ('a2specback', 'HEAO 1 A2 Spectra Background'),
 ('a2spectra', 'HEAO 1 A2 Spectra'),
 ('a3', 'HEAO 1 A3 MC LASS Catalog'),
 ('a4', 'HEAO 1 A4 Catalog of High-Energy X-Ray Sources'),
 ('a4spectra', 'HEAO 1 A4 Spectra'),
 ('aavsovsx', 'AAVSO International Variable Star Index')]

In [14]:
Xamin.query_region(hosts[35]['coord'], table='chanmaster', radius=10.*u.degree)

__row,obsid,status,name,ra,dec,lii,bii,time,detector,grating,exposure,type,pi,cycle,proposal,public_date,sequence_number,data_mode,category,class,__x_ra_dec,__y_ra_dec,__z_ra_dec
,,,,deg,deg,deg,deg,d,,,s,,,,,d,,,,,,,
object,int32,object,object,float64,float64,float64,float64,float64,object,object,float64,object,object,int16,object,int32,int32,object,object,int16,float64,float64,float64
3925,5756,archived,ClG J0405-4100,61.35125,-41.00417,245.13153,-48.09830,53670.362650463,ACIS-I,NONE,8050,GO,Vikhlinin,6,06800101,54241,800469,TE_0048A,CLUSTERS OF GALAXIES,5000,0.662272629986049,0.361814097813836,-0.656113955189586
3926,7191,archived,ClG J0405-4100,61.35125,-41.00417,245.13153,-48.09830,53874.0065162037,ACIS-I,NONE,70130,GO,Vikhlinin,6,06800101,54241,800469,TE_0048A,CLUSTERS OF GALAXIES,5000,0.662272629986049,0.361814097813836,-0.656113955189586
3954,22015,archived,DES J0420-4037,65.19500,-40.62408,244.41843,-45.20583,58613.8849652778,ACIS-S,NONE,1610,DDT,Pooley,19,19708663,58709,703835,TE_005C6,ACTIVE GALAXIES AND QUASARS,7000,0.688973265917551,0.318423306396047,-0.651093262747138
4189,4065,archived,BR 0353-3820,58.77042,-38.19508,240.99781,-50.22960,52892.1900347222,ACIS-S,NONE,4120,GO,Vignali,4,04700635,53260,700744,TE_002A2,ACTIVE GALAXIES AND QUASARS,7000,0.672029045716722,0.407469605019595,-0.618340911389703
4199,7227,archived,ClG J0350-3801,57.68292,-38.02361,240.78531,-51.09183,53813.8946875,ACIS-I,NONE,25050,GTO,Murray,6,06800018,54180,800611,TE_0048A,CLUSTERS OF GALAXIES,5000,0.665735404575944,0.421138274121598,-0.615986140399267
4200,24778,archived,Tol 0440-381,70.53379,-38.01969,241.07406,-40.97823,59261.1050462963,ACIS-S,NONE,28060,GO,Kaaret,21,21620354,59632,601463,TE_004A6,NORMAL GALAXIES,6000,0.742767106395287,0.262534755060723,-0.615932243061182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7543,17313,archived,MACSJ0416.1-2403,64.03583,-24.07794,221.10057,-44.06237,56987.7554282407,ACIS-I,NONE,63600,GO,Jones,15,15800858,56989,801418,TE_00458,CLUSTERS OF GALAXIES,5000,0.820841290424825,0.399715817949482,-0.407978971044692


In [15]:
Xamin.query_region(hosts[35]['coord'], table='rassmaster', radius=10.*u.degree)

__row,seq_id,instrument,filter,site,exposure,fits_type,ra,dec,lii,bii,start_date,end_date,dist_date,public_date,ror,index_id,proc_rev,title,qa_number,__x_ra_dec,__y_ra_dec,__z_ra_dec
,,,,,s,,deg,deg,deg,deg,d,d,d,d,,,,,,,,
object,object,object,object,object,int32,object,float64,float64,float64,float64,int32,int32,int32,int32,int32,object,int16,object,int32,float64,float64,float64
249,RS932409N00,PSPC,N,MPE,678,RFITS V4.,58.84583,-39.37500,242.8330967,-50.111609,48102,48481,51568,51615,932409,n00,3,RASS 3/24/09,91521,0.661525632163091,0.399911238900942,-0.634393284163645
250,RS932410N00,PSPC,N,MPE,791,RFITS V4.,65.76692,-39.37500,242.65413304,-44.76421475,48102,48481,51568,51615,932410,n00,3,RASS 3/24/10,91522,0.704895299360487,0.317281858837775,-0.634393284163645
251,RS932411N00,PSPC,N,MPE,619,RFITS V4.,72.69171,-39.37500,243.03591539,-39.42007611,48102,48481,51568,51615,932411,n00,3,RASS 3/24/11,91523,0.7380068041025,0.229980690725282,-0.634393284163645
301,RS932309N00,PSPC,N,MPE,566,RFITS V4.,55.63363,-33.75000,233.87868761,-52.79873073,48102,48481,51568,51615,932309,n00,3,RASS 3/23/09,91466,0.686332364813926,0.469350190360533,-0.555570233019602
302,RS932310N00,PSPC,N,MPE,717,RFITS V4.,62.17942,-33.75000,234.34806173,-47.36532686,48102,48481,51568,51615,932310,n00,3,RASS 3/23/10,91467,0.735362842028918,0.388050520867705,-0.555570233019602
303,RS932311N00,PSPC,N,MPE,718,RFITS V4.,68.72521,-33.75000,235.19210116,-41.95673421,48102,48481,51568,51615,932311,n00,3,RASS 3/23/11,91468,0.774805753485615,0.301691498965636,-0.555570233019602
304,RS932312N00,PSPC,N,MPE,659,RFITS V4.,75.27100,-33.75000,236.32984626,-36.58661656,48103,48480,51568,51615,932312,n00,3,RASS 3/23/12,91469,0.804146847721917,0.211399062158396,-0.555570233019602
357,RS932210N00,PSPC,N,MPE,628,RFITS V4.,58.96279,-28.12500,225.41647996,-49.39084197,48102,48481,51568,51615,932210,n00,3,RASS 3/22/10,91409,0.755658934200043,0.454713856918234,-0.471396736825998


In [16]:
Xamin.query_region(hosts[35]['coord'], table='xmmmaster', radius=10.*u.degree)

__row,obsid,pno,name,ra,dec,lii,bii,time,end_time,duration,scheduled_duration,estimated_exposure,pi_title,pi_fname,pi_lname,sas_version,pps_flag,pps_version,odf_date,process_date,distribution_date,public_date,xmm_revolution,process_status,mos1_num,mos1_time,mos1_mode,mos2_num,mos2_time,mos2_mode,pn_num,pn_time,pn_mode,rgs1_num,rgs1_time,rgs1_mode,rgs2_num,rgs2_time,rgs2_mode,om_num,om_time,om_mode,data_in_heasarc,subject_category,status,class,__x_ra_dec,__y_ra_dec,__z_ra_dec
,,,,deg,deg,deg,deg,d,d,s,s,s,,,,,,,d,d,d,d,,,,s,,,s,,,s,,,s,,,s,,,s,,,,,,,,
object,object,int32,object,float64,float64,float64,float64,float64,float64,int32,int32,int32,object,object,object,object,object,object,float64,float64,int32,int32,int16,object,int16,int32,object,int16,int32,object,int16,int32,object,int16,int32,object,int16,int32,object,int16,int32,object,object,object,object,int16,float64,float64,float64
3754,0303561901,30356,Tol0420-414,65.49417,-41.32250,245.40252,-44.96901,53595.4316782407,53595.5108101852,6837,4935,4935,Dr,Daniel,Rosa-Gonzalez,17.56,Y,17.56_20190403_1200,55462.0741435185,58581.3124884259,53633,53998,1040,delivered,1,6571,FF-TN (1),1,6577,FF-TN (1),1,4938,FF-TN (1),1,6821,SES (1),1,6829,SES (1),3,5597,IM-M2 (1); IM-U (1); IM-W1 (1),Y,Galaxies and Galactic Surveys,archived,6400,0.683353674298013,0.311506189241095,-0.660296637798418
3892,0744210201,74421,dummy,60.00000,-40.00000,243.71215,-49.18665,56873.4827546296,56873.4953819444,1091,--,--,Dr,Andrea,De Luca,,N,,56884.5860416667,--,--,51544,--,,0,--,,0,--,,0,--,,0,--,,0,--,,0,--,,Y,,archived,9999,0.663413948168938,0.383022221559489,-0.642787609686539
3986,0903990901,90399,LCRS B040659.9-38592,62.19207,-38.85861,241.90146,-47.53669,59648.041400463,59648.338275463,25650,22124,22124,Dr,Mirko,Krumpe,19.17,Y,19.17_20220121_1250,59653.4126388889,59654.5627314815,59662,60027,4075,delivered,1,24279,FF-ME (1),1,24258,FF-ME (1),1,22054,LW-ME (1),1,24568,SES (1),1,24202,SES (1),5,21799,IF-M2 (5),Y,"AGN, QSOs and BL-Lacs",archived,7000,0.688769909307702,0.363269014560342,-0.627400697395705
3987,0903992401,90399,LCRS B040659.9-38592,62.19207,-38.85861,241.90146,-47.53669,59647.972974537,59648.041400463,5912,--,--,Dr,Mirko,Krumpe,19.17,Y,19.17_20220121_1250,59653.4129050926,59654.5628703704,--,59662,--,,0,--,,0,--,,0,--,,0,--,,0,--,,0,--,,Y,,archived,9999,0.688769909307702,0.363269014560342,-0.627400697395705
3988,0862770501,86277,LCRS B040659.9-38592,62.19207,-38.85861,241.90146,-47.53669,59447.8321527778,59448.1377083333,26400,24684,24582,Dr,Mirko,Krumpe,19.16,Y,19.16_20210326_1200,59452.6358564815,59453.4496527778,59479,59844,3974,delivered,1,25026,SW3-ME (1),1,25005,SW3-ME (1),1,24540,SW-ME (1),1,25313,SES (1),1,25307,SES (1),7,21894,IF-M2 (7),Y,"AGN, QSOs and BL-Lacs",archived,7000,0.688769909307702,0.363269014560342,-0.627400697395705
3990,0104860101,10486,[HB89] 0420-388,65.56167,-38.74805,241.76543,-44.91026,52653.8646064815,52654.1292476852,22865,20998,16714,Prof,Keith,Mason,17.56,Y,17.56_20190403_1200,55506.2251157407,58581.126400463,52689,53054,568,delivered,1,22623,FF-TN (1),1,22624,FF-TN (1),1,16716,FF-TN (1),1,22865,SES (1),1,22864,SES (1),4,17240,IM-B (1); IM-U (1); IM-GRISM2 (2),Y,"AGN, QSOs and BL-Lacs",archived,7200,0.710031761511101,0.322657607293509,-0.625896929295129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5594,0801780301,80178,SWIFTJ0503.7-2819,75.95541,-28.38577,230.04685,-34.82523,58184.4366782407,58184.7607523148,28000,25039,25039,Dr,Domitilla,de Martino,17.56,Y,17.56_20190403_1200,58204.6066435185,58672.5334953704,58218,58582,3341,delivered,1,26655,FF-TN (1),1,26628,FF-TN (1),1,25041,FF-TN (1),1,26921,SES (1),1,26905,SES (1),10,21603,IF-V (10),Y,"White Dwarf Binaries, Neutron Star Binaries, Cataclysmic Variables, ULXs and Black Holes",archived,9999,0.853467946342585,0.213499090415233,-0.475405724573904


In [19]:
Xamin.query_region(hosts[35]['coord'], table='erosmaster', radius=10.*u.degree)

__row,obsid,file_id,name,ra,dec,lii,bii,obs_type,category_code,__x_ra_dec,__y_ra_dec,__z_ra_dec
,,,,deg,deg,deg,deg,,,,,
object,int32,object,object,float64,float64,float64,float64,object,object,float64,float64,float64


Alright, this seems to be working as expected for everything except eROSITA.

So what we want to do is for each host, query the relevant "master" catalog with an area that's about matched to whatever the main detector's FOV is (generously) to find which have any observations.  Only from those do we search for actual sources.

For eROSITA it turns out it's a totally separate source, so we skip that and use the vo cone search directly from eROSITA instead of HEASARC.  Similarly, for the ROSAT rassmaster, the all-sky-survey is supposed to be, well, an all-sky survey so we skip it.

In [20]:
mission_to_rad = {'xmmmaster': 15*u.arcmin, # " telescope's field of view (FOV) of 30 arcmin and in th"
                  'chanmaster':8*u.arcmin, # this is ~ half ASIC's FOV
                  'rosmaster':2.5*u.deg,  #"5 degree diameter"
                  'rassmaster':2.5*u.deg,  #"5 degree diameter"
                }

In [30]:
host_results = []
    
for host in tqdm(hosts):
    reses = {}
    for tabname, rad in mission_to_rad.items():
        reses[tabname] = Xamin.query_region(host['coord'], table=tabname, radius=rad)
    host_results.append(reses)
            


  0%|          | 0/300 [00:00<?, ?it/s]

Better parallel:

In [ ]:
pres = Path('heasarcXamin_master_results.pickle')

results = []
if pres.exists():
    print(f'{pres} already exists, not overwriting, loading')
    with pres.open('rb') as f:
        results.extend(load(f))
else:
    with Pool(1) as p:
        reses = []
        for host in hosts:
            for tabname, rad in mission_to_rad.items():
                ares = p.apply_async(Xamin.query_region, (host['coord'],), kwds={'table':tabname, 'radius':rad})
                reses.append((host['HOSTID'], tabname, ares))
        t = tqdm(total=len(reses))
        while t.n < t.total:
            nrdy = sum([r[2].ready() for r in reses])
            if nrdy > t.n:
                t.update(nrdy - t.n)
    
        for r in reses:
            results.append((r[0], r[1], r[2].get()))

    print(f'writing to {pres}')
    with pres.open('bw') as f:
        dump(results, f)

  0%|          | 0/1200 [00:00<?, ?it/s]

Process ForkPoolWorker-14:
Traceback (most recent call last):
  File "/Users/etollerud/miniforge3/envs/py311/lib/python3.11/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/etollerud/miniforge3/envs/py311/lib/python3.11/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/etollerud/miniforge3/envs/py311/lib/python3.11/site-packages/multiprocess/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/etollerud/miniforge3/envs/py311/lib/python3.11/site-packages/multiprocess/queues.py", line 370, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/etollerud/miniforge3/envs/py311/lib/python3.11/site-packages/dill/_dill.py", line 303, in loads
    return load(file, ignore, **kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/etollerud/miniforge3/envs/py311/lib/python3.11/site-packages/dill/_dill.py", line 2